In [1]:
from sklearn.metrics import mean_squared_error
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from ase.visualize import view
import ase.io
from amp import Amp
from amp.model.neuralnetwork import NeuralNetwork
from amp.model import LossFunction
import operator
import amptorch
import copy
import matplotlib
from skorch import NeuralNetRegressor
from skorch.dataset import CVSplit
from skorch.callbacks import Checkpoint, EpochScoring
from skorch.callbacks.lr_scheduler import LRScheduler
import skorch.callbacks.base
from amptorch.gaussian import SNN_Gaussian
from amptorch.model import BPNN, CustomMSELoss
from amptorch.data_preprocess import AtomsDataset, factorize_data, collate_amp, TestDataset
from amptorch.skorch_model import AMP
from amptorch.skorch_model.utils import target_extractor, energy_score, forces_score
from amptorch.analysis import parity_plot
from torch.utils.data import DataLoader
from torch.nn import init
from skorch.utils import to_numpy
import matplotlib.pyplot as plt

In [2]:
def Select_image(images):
    train_image = []
    test_image = []
    for index,image in enumerate(images[:]):
        if index-index//3*3 == 0:
            test_image.append(image)
        else:
            train_image.append(image)
    print(len(train_image), len(test_image))
    return train_image, test_image


In [3]:
def run_model(assignments, numberi=0):
    class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
        def on_train_end(self, net, X, y):
            net.load_params('valid_best_params.pt')
    out_file = open('log-sigopt','a')
    # LR_schedule = LRScheduler("CosineAnnealingLR", T_max=assignments["T_max"])
    cp = Checkpoint(monitor='valid_loss_best', fn_prefix='valid_best_')
    load_best_valid_loss = train_end_load_best_valid_loss()
    images = ase.io.read('./traj_taged_adsorptionenergy.traj', index=':')
    Gs = {}
    Gs["G2_etas"] = np.logspace(np.log10(0.05), np.log10(5.0), num=4)
    Gs["G2_rs_s"] = [0] * 4
    Gs["G4_etas"] = [0.005]
    Gs["G4_zetas"] = [1.0]
    Gs["G4_gammas"] = [+1.0, -1]
    Gs["cutoff"] = 6.5
    forcetraining = False
    label = "sigopt-zeolite"
    train_images, test_images = Select_image(images)

    DFT_energies_test = [image.get_potential_energy() for image in test_images]
    DFT_energies_train = [image.get_potential_energy() for image in train_images]
    training_data = AtomsDataset(train_images, SNN_Gaussian, Gs, forcetraining=forcetraining,
            label=label, cores=1, delta_data=None, specific_atoms=True)

    unique_atoms = training_data.elements
    fp_length = training_data.fp_length
    device = "cpu"
    torch.set_num_threads(1)
    optimizer = optim.LBFGS
    batch_size = len(training_data)
    net = NeuralNetRegressor(
        module=BPNN(
            unique_atoms,
            [fp_length, assignments["hidden_layers"], assignments["num_nodes"]],
            device,
            forcetraining=forcetraining,
        ),
        criterion=CustomMSELoss,
        criterion__force_coefficient=0,
        optimizer=optimizer,
        #optimizer=torch.optim.LBFGS,
        lr=assignments["learning_rate"],
        #lr=1e-1,
        batch_size=batch_size,
        max_epochs=assignments["epochs"],
        iterator_train__collate_fn=collate_amp,
        iterator_train__shuffle=False,
        iterator_valid__collate_fn=collate_amp,
        iterator_valid__shuffle=False,
        device=device,
        train_split=CVSplit(cv=0.2),
        callbacks=[
            EpochScoring(
                energy_score,
                on_train=True,
                use_caching=True,
                target_extractor=target_extractor,
            ),
             cp,
             load_best_valid_loss,
            # LR_schedule
        ],
    )
    calc = AMP(training_data, net, label,specific_atoms=True)
    calc.train(overwrite=True)
    for image in test_images:
        image.set_calculator(calc)
    for image in train_images:
        image.set_calculator(calc)
    pred_energies_test = [image.get_potential_energy() for image in test_images]
    pred_energies_train = [image.get_potential_energy() for image in train_images]
    for e_t in pred_energies_test:
        if np.isnan(e_t):
            e_t = 1e10
    for e_tr in pred_energies_train:
        if np.isnan(e_tr):
            e_tr =1e10
    energy_rmse_test = mean_squared_error(pred_energies_test, DFT_energies_test)
    energy_rmse_train = mean_squared_error(pred_energies_train, DFT_energies_train)
    print('***************', file=out_file)
    print('No. ', numberi, file=out_file)
    print('**RMSE_TEST:', energy_rmse_test,file=out_file)
    print('**RMSE_Train:', energy_rmse_train,file=out_file)
    print('Energies:::::::', file=out_file)
    print('train data',file=out_file)
    for i in range(len(train_images)):
        print('(',DFT_energies_train[i],';',pred_energies_train[i],') ', end='',file=out_file)
    print('****',file=out_file)
    print('test data', file=out_file)
    for i in range(len(test_images)):
        print('(',DFT_energies_test[i],';',pred_energies_test[i],') ', end='',file=out_file)
    print('****',file=out_file)
    fig = plt.figure('ML vs DFT')
    ax = fig.add_subplot(111)
    ax.scatter(DFT_energies_train, pred_energies_train, c='b')
    ax.scatter(DFT_energies_test, pred_energies_test, c='r')
    ax.set_xlabel('E_DFT')
    ax.set_ylabel('E_ML')
    figure_name = './DFTvsML'+ str(numberi)+ '.png'
    fig.savefig(figure_name, bbox_inches='tight')
    fig = 0
    return energy_rmse_test, energy_rmse_train

In [4]:
assignments = {
        'epochs': 995,
        'learning_rate': 0.02315667835660472,
        'hidden_layers': 7,
        'num_nodes': 36
        }

In [5]:
run_model(assignments)

720 361
Calculating fingerprints...
Fingerprints Calculated!
  epoch    energy_score    train_loss    valid_loss    cp     dur
-------  --------------  ------------  ------------  ----  ------
      1          0.6139      343.9013       13.0896     +  1.9110
      2          0.5810      308.0504       19.9706        1.8890
      3          0.5512      277.2128       25.9631        1.9574
      4          0.5105      237.8072       19.2430        1.8756
      5          0.4880      217.2871       16.2464        1.9177
      6          0.4775      208.0200       14.3731        1.9798
      7          0.4494      184.2796       15.3259        1.9570
      8          0.3982      144.6816       16.3194        1.9870
      9          0.3592      117.7025       14.9146        1.8919
     10          0.3314      100.2286       13.5133        1.9860
     11          0.3177       92.0703       11.0497     +  2.0433
     12          0.2999       82.0531        7.9993     +  1.9992
     13        

     96          0.0507        2.3475       11.1698        1.9710
     97          0.0505        2.3313       10.9648        1.9589
     98          0.0502        2.3018       10.7024        1.8830
     99          0.0497        2.2579       10.3863        1.9563
    100          0.0492        2.2097       10.1128        1.9727
    101          0.0488        2.1728        9.9165        1.9368
    102          0.0485        2.1446        9.6813        1.9899
    103          0.0480        2.1061        9.4619        1.9196
    104          0.0475        2.0596        9.3107        1.9783
    105          0.0471        2.0256        9.2312        2.0197
    106          0.0469        2.0038        9.1780        1.9709
    107          0.0467        1.9888        9.1445        1.9620
    108          0.0465        1.9744        9.1251        1.9196
    109          0.0463        1.9566        9.1254        1.9416
    110          0.0461        1.9381        9.1416        1.9755
    111   

    193          0.0302        0.8336        8.5986        1.9483
    194          0.0300        0.8224        8.6299        1.9223
    195          0.0299        0.8141        8.6348        1.9834
    196          0.0297        0.8058        8.6115        1.9498
    197          0.0295        0.7947        8.5656        1.9738
    198          0.0293        0.7852        8.4883        1.9122
    199          0.0292        0.7757        8.4205        1.9403
    200          0.0290        0.7674        8.3558        1.9138
    201          0.0289        0.7605        8.3159        1.9829
    202          0.0288        0.7545        8.2995        1.9984
    203          0.0286        0.7487        8.2791        1.8923
    204          0.0285        0.7421        8.2102        1.9963
    205          0.0283        0.7319        8.1472        1.9860
    206          0.0281        0.7198        8.0982        1.9948
    207          0.0279        0.7094        8.0760        1.8829
    208   

    291          0.0172        0.2694       13.0548        1.9304
    292          0.0171        0.2670       13.1150        1.8729
    293          0.0170        0.2634       13.1469        1.9728
    294          0.0169        0.2615       13.1755        1.9742
    295          0.0169        0.2604       13.2031        1.9793
    296          0.0169        0.2596       13.2407        1.9194
    297          0.0168        0.2586       13.3272        1.9422
    298          0.0168        0.2568       13.4459        1.9612
    299          0.0167        0.2547       13.5782        1.9524
    300          0.0166        0.2521       13.7435        1.9712
    301          0.0165        0.2491       13.9301        1.9005
    302          0.0164        0.2462       14.1412        1.9751
    303          0.0163        0.2434       14.2818        1.9715
    304          0.0163        0.2411       14.4808        1.9836
    305          0.0162        0.2394       14.6521        1.9464
    306   

    389          0.0118        0.1281       28.3987        1.9822
    390          0.0118        0.1275       28.5668        1.8889
    391          0.0118        0.1266       28.7455        1.9633
    392          0.0117        0.1255       28.8817        1.9368
    393          0.0117        0.1247       28.9989        1.9343
    394          0.0117        0.1240       29.1049        1.9570
    395          0.0116        0.1234       29.2284        1.9627
    396          0.0116        0.1227       29.3724        1.9020
    397          0.0115        0.1216       29.5056        1.9240
    398          0.0115        0.1204       29.5869        1.9231
    399          0.0114        0.1192       29.6364        1.9630
    400          0.0114        0.1184       29.6440        1.8788
    401          0.0114        0.1177       29.6463        1.9669
    402          0.0113        0.1172       29.6627        1.9622
    403          0.0113        0.1167       29.7173        1.9537
    404   

...

    683          0.0049        0.0221       31.9973        1.9607
    684          0.0049        0.0221       32.0304        1.9717
    685          0.0049        0.0220       32.0740        1.8963
    686          0.0049        0.0219       32.1242        1.9220
    687          0.0049        0.0218       32.1790        1.9619
    688          0.0049        0.0217       32.2286        1.9662
    689          0.0049        0.0215       32.2679        1.9701
    690          0.0048        0.0214       32.2845        1.8971
    691          0.0048        0.0212       32.2848        1.8700
    692          0.0048        0.0211       32.2776        1.9569
    693          0.0048        0.0210       32.2652        1.9559
    694          0.0048        0.0209       32.2550        2.0008
    695          0.0048        0.0208       32.2519        1.8746
    696          0.0048        0.0207       32.2623        1.8546
    697          0.0048        0.0206       32.2809        1.9627
    698   

    781          0.0039        0.0140       33.4304        1.9628
    782          0.0039        0.0139       33.4323        1.8771
    783          0.0039        0.0138       33.4394        1.9445
    784          0.0039        0.0138       33.4468        1.9323
    785          0.0039        0.0137       33.4516        1.9644
    786          0.0039        0.0137       33.4503        1.8665
    787          0.0039        0.0136       33.4425        1.8604
    788          0.0039        0.0135       33.4351        1.9271
    789          0.0038        0.0135       33.4302        1.9682
    790          0.0038        0.0134       33.4308        1.9798
    791          0.0038        0.0134       33.4312        1.9060
    792          0.0038        0.0133       33.4336        1.9589
    793          0.0038        0.0133       33.4389        1.9563
    794          0.0038        0.0132       33.4455        1.9313
    795          0.0038        0.0132       33.4524        1.9772
    796   

    879          0.0031        0.0087       32.3938        1.9166
    880          0.0031        0.0087       32.3966        1.9301
    881          0.0031        0.0087       32.4001        1.9800
    882          0.0031        0.0086       32.4018        1.9851
    883          0.0031        0.0086       32.3982        2.0110
    884          0.0031        0.0085       32.3918        1.8669
    885          0.0030        0.0085       32.3852        1.9340
    886          0.0030        0.0085       32.3790        1.9855
    887          0.0030        0.0084       32.3718        1.9827
    888          0.0030        0.0084       32.3652        1.9354
    889          0.0030        0.0084       32.3602        1.9148
    890          0.0030        0.0083       32.3601        1.9690
    891          0.0030        0.0083       32.3602        1.9594
    892          0.0030        0.0083       32.3608        1.9056
    893          0.0030        0.0082       32.3601        1.9593
    894   

    977          0.0025        0.0058       32.8847        1.9691
    978          0.0025        0.0058       32.9011        1.9825
    979          0.0025        0.0058       32.9150        1.9332
    980          0.0025        0.0058       32.9263        1.9078
    981          0.0025        0.0057       32.9376        1.9545
    982          0.0025        0.0057       32.9491        1.9819
    983          0.0025        0.0057       32.9637        1.9806
    984          0.0025        0.0057       32.9787        1.9761
    985          0.0025        0.0057       32.9957        2.0031
    986          0.0025        0.0057       33.0161        1.9361
    987          0.0025        0.0057       33.0399        1.9568
    988          0.0025        0.0056       33.0660        1.9947
    989          0.0025        0.0056       33.0929        1.9848
    990          0.0025        0.0056       33.1178        1.9600
    991          0.0025        0.0056       33.1427        1.8651
    992   

(3.1543664925188364, 1.7374764715340063)